## First we clean our dataset and create our dataset according our requirement :

In [1]:
# required libraries
import psycopg2
import pandas as pd

In [2]:
df_data_country = pd.read_csv("Wealth-AccountsCountry.csv")

In [3]:
df_data_country.columns

Index(['Code', 'Long Name', 'Income Group', 'Region', 'Lending category',
       'Other groups', 'Currency Unit', 'Latest population census',
       'Latest household survey', 'Special Notes',
       'National accounts base year', 'National accounts reference year',
       'System of National Accounts', 'SNA price valuation',
       'Alternative conversion factor', 'PPP survey years',
       'Balance of Payments Manual in use', 'External debt Reporting status',
       'System of trade', 'Government Accounting concept',
       'IMF data dissemination standard',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', '2-alpha code', 'WB-2 code',
       'Table Name', 'Short Name'],
      dtype='object')

In [4]:
Account_country = df_data_country[["Code", "Long Name", "Income Group", "Region", "Currency Unit"]]

In [5]:
Account_country = Account_country.rename(columns = {"Code":"Country Code"})

In [6]:
Account_country = Account_country.drop_duplicates(subset = ["Country Code"])

In [8]:
Account_country.head()

,Country Code,Long Name,Income Group,Region,Currency Unit
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,Albanian lek
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,Argentine peso
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,Armenian dram
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,Australian dollar
4,AUT,Republic of Austria,High income,Europe & Central Asia,Euro


In [10]:
df_data = pd.read_csv("Wealth-AccountData.csv")

In [11]:
df_data.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [12]:
clean_AccountData = df_data[["Country Name","Country Code","Series Name","Series Code","1995 [YR1995]","2000 [YR2000]","2005 [YR2005]",\
                             "2010 [YR2010]","2014 [YR2014]"]]

In [13]:
Account_Data = clean_AccountData.rename(columns={"Series Name":"Indicator Name", "1995 [YR1995]":"1995", "2000 [YR2000]":"2000", \
                                                 "2005 [YR2005]":"2005", "2010 [YR2010]":"2010", "2014 [YR2014]":"2014"})

In [14]:
Account_Data.head()

,Country Name,Country Code,Indicator Name,Series Code,1995,2000,2005,2010,2014
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


In [15]:
df_series = pd.read_csv("Wealth-AccountSeries.csv")

In [16]:
df_series.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [17]:
Account_Series = df_series[["Code", "Topic", "Indicator Name", "Long definition", "Source"]]

In [18]:
Account_Series.head()

,Code,Topic,Indicator Name,Long definition,Source
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...


### Now we create functions for our database design

In [19]:
def create_database():
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur= conn.cursor()
    # now we create database with name accounts
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")

    # close the connection
    conn.close()
    # now we create new connection :
    conn = psycopg2.connect("host=localhost dbname=accounts user=postgres password=root")
    cur = conn.cursor()

    return cur, conn

In [20]:
def drop_tables(cur, conn):
    for query in drop_table_queries :
        cur.execute(query)
        conn.commit

In [21]:
def create_tables(cur, conn):
    for query in create_table_queries :
        cur.execute(query)
        conn.commit

In [22]:
cur, conn = create_database()

In [23]:
# Now we creates table of our database :

In [24]:
country_table_create = ("""CREATE TABLE IF NOT EXISTS accountcountry(country_code VARCHAR PRIMARY KEY, long_name VARCHAR, income_group VARCHAR, \
                            Region VARCHAR, currency_unit VARCHAR)""")
cur.execute(country_table_create)
conn.commit()

In [25]:
data_table_create = ("""CREATE TABLE IF NOT EXISTS accountdata(country_name VARCHAR, country_code VARCHAR, indicator_name VARCHAR, \
                                                            series_code VARCHAR, year_1995 VARCHAR, year_2000 VARCHAR, year_2005 VARCHAR, year_2010 VARCHAR, year_2014 VARCHAR)""")
cur.execute(data_table_create)
conn.commit()

In [26]:
series_table_execute = ("""CREATE TABLE IF NOT EXISTS accountseries (code VARCHAR, topic VARCHAR, indicator_name VARCHAR, long_definition VARCHAR, \
                                                                        source VARCHAR) """)
cur.execute(series_table_execute)
conn.commit()

In [27]:
# now we wants to insert the data in tables

In [28]:
country_insert = ("""INSERT INTO accountcountry (country_code, long_name, income_group, Region, currency_unit) VALUES(%s, %s, %s, %s, %s)""")

In [29]:
for i, row in Account_country.iterrows():
    cur.execute(country_insert, list(row))

conn.commit()

In [30]:
data_insert = ("""INSERT INTO accountdata (country_name, country_code, indicator_name, series_code, year_1995, year_2000, year_2005, year_2010, year_2014) \
                                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)""")

In [31]:
for i, row in Account_Data.iterrows():
    cur.execute(data_insert, list(row))
conn.commit()

In [32]:
series_insert = ("""INSERT INTO accountseries (code, topic, indicator_name, long_definition, source) VALUES(%s, %s, %s, %s, %s)""")

In [33]:
for i, row in Account_Series.iterrows():
    cur.execute(series_insert, list(row))

conn.commit()